<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_6_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset with brain atlas B for continuous prediction tasks: for second set of 5 small
programming tasks: Oasis dataset provided directly by the nilearn package
Can be retrieved by downloading nilearn.datasets.fetch_oasis_vbm(n_subjects=100) ,
roughly 900 MB, structural brain scans (so-called voxel-based morphometry) to predict of
continuous outcome given a subject’s age from structural brain scans

6- analogous to task 1, but implementing a Bayesian hierarchical regression (with
continuous outcome variable) to predict subject age (zscored).

In [ ]:
%%capture
!pip install nilearn
!pip install pymc

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
from sklearn.preprocessing import StandardScaler
import nibabel as nib

NOTE: for this exercise I used PyMC version 5 instead of PyMC3 because the model ran better on the former package. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc as pm
import scipy.stats as stats
import arviz as az
from pymc import Bernoulli, Model, HalfCauchy
from pymc import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()
atlas_nii = index_img(crad['scorr_mean'], 10) # we will set this to 10 for these exercises in order to have 106 rois, which is close to the 100 rois in Exercises 1-5
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = StandardScaler().fit_transform(brain_data.ext_vars.age[:, None])[:, 0] # gives subject age on standard units after z-scoring

In [ ]:
# Since there is no information about brain side in this dataset, we will assume it is 50-50 like in the yeo dataset 
with pm.Model() as model:
    # We define a HalfCauchy hyperprior for variance component of lower-level region slopes
    hc = pm.HalfCauchy('hc', beta=1, shape=106)

    # We define binary hyperprior category based on brain hemisphere (side) with left and right having equal proportion, in line with out assumption
    brain_side = pm.Categorical('brain_side', p=np.array([0.5, 0.5]))
    
    # We define the intercept term as a normal distribution with priors mu=0 and sigma=1
    intercept = pm.Normal('intercept', mu=0, sigma=1)
    
    # We define the bottom-level region slopes (or betas) as a normal distribution with priors (mu=0, sigma=hyperprior)
    beta_coefficients = pm.Normal('beta_coefficients', mu=0, sigma=hc, shape=106)
    
    # We establish the desired linear model using input_variables as our "X"es and the required model parameters
    linear_model = intercept + input_variables @ beta_coefficients + brain_side
    
    # We define the likelihood as another Normal distribution with mean equal to probabilities and sigma=1
    likelihood = pm.Normal('likelihood', mu=linear_model, sigma=1, observed=output_variable)

    # We sample from the posterior distribution to estimate the unseen variables using 500 MCMC draws.
    trace = pm.sample(500, return_inferencedata=True)
    # This time we will call the arviz package to creat the summary table 
    stats = az.summary(trace, hdi_prob=0.9)

/usr/local/lib/python3.8/dist-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


In [ ]:
# Now we're ready to take a look at the summary table
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=214)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
brain_side,0.000,0.000,0.000,0.000,0.000,0.000,1000.0,1000.0,NaN
intercept,0.010,0.103,-0.166,0.161,0.011,0.008,111.0,407.0,1.04
beta_coefficients[0],0.027,0.242,-0.346,0.472,0.014,0.010,351.0,304.0,1.05
beta_coefficients[1],-0.077,0.308,-0.646,0.346,0.021,0.015,190.0,329.0,1.01
beta_coefficients[2],0.046,0.283,-0.368,0.544,0.023,0.016,151.0,240.0,1.03
...,...,...,...,...,...,...,...,...,...
hc[101],0.754,1.681,0.002,1.999,0.119,0.084,45.0,75.0,1.04
hc[102],0.797,0.908,0.003,1.801,0.039,0.028,433.0,428.0,1.03
hc[103],0.669,1.208,0.004,1.268,0.060,0.042,326.0,289.0,1.03
hc[104],0.671,1.062,0.002,1.449,0.068,0.048,237.0,242.0,1.02
